In [1]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install seaborn 

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install tensorflow

   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/390.3 MB ? eta -:--:--
   ---------------------------------------- 1.8/390.3 MB 7.7 MB/s eta 0:00:51
    --------------------------------------- 6.0/390.3 MB 13.7 MB/s eta 0:00:29
    --------------------------------------- 7.6/390.3 MB 12.1 MB/s eta 0:00:32
    --------------------------------------- 9.4/390.3 MB 11.5 MB/s eta 0:00:34
   - -------------------------------------- 11.0/390.3 MB 10.4 MB/s eta 0:00:37
   - -------------------------------------- 12.3/390.3 MB 9.9 MB/s eta 0:00:39
   - -------------------------------------- 13.6/390.3 MB 9.0 MB/s eta 0:00:42
   - -------------------------------------- 15.5/390.3 MB 8.9 MB/s eta 0:00:42
   - -------------------------------------- 17.6/390.3 MB 9.1 MB/s eta 0:00:42
   - -------------------------------------- 18.4/390.3 MB 8.7 MB/s eta 0:00:43
   - -------------------------------------- 19.1/390.3 MB 8.3 MB/s e

In [7]:
pip install sklearn

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler


In [9]:
import os
print(os.getcwd())

c:\Users\user\Desktop\CodeAlpha_STOCK_PREDICTION
